In [6]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

In [8]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="You are a a geography expert. And you only reply in Itailian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Rome and Naples? Also, what is your name?"),
]

chat.predict_messages(messages)



AIMessage(content='Ciao! La distanza tra Roma e Napoli è di circa 225 chilometri. Come ti chiami?')

In [13]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("Whate is the distance between {country_a} and {country_b}?")
prompt = template.format(country_a="Italy", country_b="France")

chat.predict(prompt)



'The distance between Italy and France varies depending on the specific locations being compared. However, the approximate distance between the two countries is around 500-600 kilometers (310-370 miles) when measuring from the northern border of Italy to the southern border of France.'

In [20]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?"),
])

prompt = template.format_messages(language="Greek", country_a="Italy", country_b="France", name="Paolo")


chat.predict_messages(prompt)


AIMessage(content='Καλησπέρα, το όνομά μου είναι Παναγιώτης. Η απόσταση μεταξύ Ιταλίας και Γαλλίας είναι περίπου 1000 χιλιόμετρα.')

In [22]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        items = text.strip().split(",")
        return [item.strip() for item in items]

parser = CommaOutputParser()
parser.parse("Hello, world!")




['Hello', 'world!']

In [31]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(question="What are the colors??", max_items=10)
result = chat.predict_messages(prompt)

parser = CommaOutputParser()
parser.parse(result.content)


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [34]:
chain = template | chat | CommaOutputParser()

chain.invoke({"question": "What are the colors?", "max_items": 10})


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class international chef. You create asy to follow recipies for any type of cuisin with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat 

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't Know how to replace it."),
    ("human", "{recipe}"),
])

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chef_chain

final_chain.invoke({"cuisine": "indian"})



NameError: name 's' is not defined